In [1]:
#Loading Libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import re,string,unicodedata
from keras.preprocessing import text, sequence
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,f1_score
from sklearn.model_selection import train_test_split
from string import punctuation
from nltk import pos_tag
from nltk.corpus import wordnet
import keras
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM,Dropout
from keras.callbacks import ReduceLROnPlateau
import tensorflow as tf

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import  LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import learning_curve
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn import svm
from gensim.models.word2vec import Word2Vec
import pickle
import warnings
warnings.filterwarnings("ignore")

Referenced notebooks for Modelling : (other refrences on fake news in notes)

https://www.kaggle.com/madz2000/nlp-using-glove-embeddings-99-87-accuracy 

https://www.kaggle.com/jaskaransingh/fake-news-classification-bert-roberta

https://towardsdatascience.com/light-on-math-ml-intuitive-guide-to-understanding-glove-embeddings-b13b4f19c010

https://www.kaggle.com/foolofatook/news-classification-using-bert  

https://www.kaggle.com/alfarias/huffpost-news-classification-with-distilbert

https://github.com/aramakus/ML-and-Data-Analysis/blob/master/RoBERTa%20for%20text%20classification.ipynb

https://towardsdatascience.com/fine-tuning-bert-and-roberta-for-high-accuracy-text-classification-in-pytorch-c9e63cf64646

https://towardsdatascience.com/fake-news-classification-with-bert-afbeee601f41

https://medium.com/swlh/detecting-fake-news-with-python-and-machine-learning-f78421d29a06

https://towardsdatascience.com/getting-real-with-fake-news-d4bc033eb38a

In [2]:
df = pd.read_csv("/Users/gopalika14/Desktop/fakenn.csv")
print(df.shape)
df.head()

(17326, 8)


,id,date,speaker,statement,sources,paragraph_based_content,fullText_based_content,label_fnn
0,1636,2010-03-28T17:45:34-04:00,Charlie Crist,"Rubio's tax swap proposal ""would have been a m...",['http://blogs.tampabay.com/buzz/files/040307l...,['Gov. Charlie Crist launched what amounts to ...,Gov. Charlie Crist launched what amounts to a ...,fake
1,4352,2011-08-29T06:00:00-04:00,Bobby Scott,"""The estimated savings of this (debt ceiling) ...",['http://www.bobbyscott.house.gov/index.php?op...,"['U.S. Rep. Bobby Scott, D-3rd, was not please...","U.S. Rep. Bobby Scott, D-3rd, was not pleased ...",real
2,16471,2019-02-12T17:35:38-05:00,Wisconsin Republican Legislative leaders,"Foxconn has already ""made a positive impact ac...",['https://www.wispolitics.com/2019/sen-fitzger...,"[""Amid reports questioning Foxconn Technology ...",Amid reports questioning Foxconn Technology Gr...,fake
3,1557,2010-03-05T18:24:02-05:00,Dave Aronberg,"Says Gov. Charlie Crist has called him ""a rock...","['http://www.davearonberg.com/about', 'http://...","[""State Sen. Dave Aronberg, a Democratic candi...","State Sen. Dave Aronberg, a Democratic candida...",real
4,12826,2016-07-29T18:09:31-04:00,Jeannette Vaught,"""Only five Texas counties account for almost 9...",['http://www.mystatesman.com/news/news/opinion...,['From the citrus of the Rio Grande Valley to ...,From the citrus of the Rio Grande Valley to th...,fake


In [3]:
df['DATE'] = pd.to_datetime(df['date'],utc=True, errors='coerce')
df['MONTH'] = df['DATE'].dt.month
df['year'] = df['DATE'].dt.year
df = df.drop(["date"], axis =1)
indexNames = df[ df['label_fnn'] == "label_fnn" ].index
# Delete these row indexes from dataFrame
df.drop(indexNames , inplace=True)
df.head()

,id,speaker,statement,sources,paragraph_based_content,fullText_based_content,label_fnn,DATE,MONTH,year
0,1636,Charlie Crist,"Rubio's tax swap proposal ""would have been a m...",['http://blogs.tampabay.com/buzz/files/040307l...,['Gov. Charlie Crist launched what amounts to ...,Gov. Charlie Crist launched what amounts to a ...,fake,2010-03-28 21:45:34+00:00,3.0,2010.0
1,4352,Bobby Scott,"""The estimated savings of this (debt ceiling) ...",['http://www.bobbyscott.house.gov/index.php?op...,"['U.S. Rep. Bobby Scott, D-3rd, was not please...","U.S. Rep. Bobby Scott, D-3rd, was not pleased ...",real,2011-08-29 10:00:00+00:00,8.0,2011.0
2,16471,Wisconsin Republican Legislative leaders,"Foxconn has already ""made a positive impact ac...",['https://www.wispolitics.com/2019/sen-fitzger...,"[""Amid reports questioning Foxconn Technology ...",Amid reports questioning Foxconn Technology Gr...,fake,2019-02-12 22:35:38+00:00,2.0,2019.0
3,1557,Dave Aronberg,"Says Gov. Charlie Crist has called him ""a rock...","['http://www.davearonberg.com/about', 'http://...","[""State Sen. Dave Aronberg, a Democratic candi...","State Sen. Dave Aronberg, a Democratic candida...",real,2010-03-05 23:24:02+00:00,3.0,2010.0
4,12826,Jeannette Vaught,"""Only five Texas counties account for almost 9...",['http://www.mystatesman.com/news/news/opinion...,['From the citrus of the Rio Grande Valley to ...,From the citrus of the Rio Grande Valley to th...,fake,2016-07-29 22:09:31+00:00,7.0,2016.0


In [4]:
#for square brackets and airqoutes removal
df['sources'] = df['sources'].apply(lambda x: x.replace('[','').replace(']',''))
df['paragraph_based_content'] = df['paragraph_based_content'].apply(lambda x: x.replace('[','').replace(']',''))
df['statement'] = df['statement'].apply(lambda x: x.replace('"', ''))
df['sources'] = df['sources'].str.replace("'" , "" )
df['paragraph_based_content'] = df['paragraph_based_content'].str.replace("'" , "" )
df['paragraph_based_content'] = df['paragraph_based_content'].apply(lambda x: x.replace('"', ''))
df.head()

,id,speaker,statement,sources,paragraph_based_content,fullText_based_content,label_fnn,DATE,MONTH,year
0,1636,Charlie Crist,Rubio's tax swap proposal would have been a ma...,http://blogs.tampabay.com/buzz/files/040307lt-...,Gov. Charlie Crist launched what amounts to a ...,Gov. Charlie Crist launched what amounts to a ...,fake,2010-03-28 21:45:34+00:00,3.0,2010.0
1,4352,Bobby Scott,The estimated savings of this (debt ceiling) d...,http://www.bobbyscott.house.gov/index.php?opti...,"U.S. Rep. Bobby Scott, D-3rd, was not pleased ...","U.S. Rep. Bobby Scott, D-3rd, was not pleased ...",real,2011-08-29 10:00:00+00:00,8.0,2011.0
2,16471,Wisconsin Republican Legislative leaders,Foxconn has already made a positive impact acr...,https://www.wispolitics.com/2019/sen-fitzgeral...,Amid reports questioning Foxconn Technology Gr...,Amid reports questioning Foxconn Technology Gr...,fake,2019-02-12 22:35:38+00:00,2.0,2019.0
3,1557,Dave Aronberg,Says Gov. Charlie Crist has called him a rock ...,"http://www.davearonberg.com/about, http://miam...","State Sen. Dave Aronberg, a Democratic candida...","State Sen. Dave Aronberg, a Democratic candida...",real,2010-03-05 23:24:02+00:00,3.0,2010.0
4,12826,Jeannette Vaught,Only five Texas counties account for almost 90...,http://www.mystatesman.com/news/news/opinion/v...,From the citrus of the Rio Grande Valley to th...,From the citrus of the Rio Grande Valley to th...,fake,2016-07-29 22:09:31+00:00,7.0,2016.0


# BASIC MODELLING - Full based content

In [5]:
X=df['fullText_based_content']
y=df['label_fnn']

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
#Random Forest Classifier
pp=Pipeline([
    ('bow',CountVectorizer()),
    ('tfidf',TfidfTransformer()),
    ('classifier',RandomForestClassifier())
    ])

In [8]:
pp.fit(X_train,y_train)

Pipeline(steps=[('bow', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('classifier', RandomForestClassifier())])

In [9]:
predictions=pp.predict(X_test)

In [10]:
print(confusion_matrix(y_test, predictions))
print('\n')
print(classification_report(y_test, predictions))

[[1018  674]
 [ 376 1397]]


              precision    recall  f1-score   support

        fake       0.73      0.60      0.66      1692
        real       0.67      0.79      0.73      1773

    accuracy                           0.70      3465
   macro avg       0.70      0.69      0.69      3465
weighted avg       0.70      0.70      0.69      3465



In [11]:
#Multinomial Naive Bayes
pp2=Pipeline([
    ('bow',CountVectorizer()),
    ('tfidf',TfidfTransformer()),
    ('classifier',MultinomialNB())
    ])

In [12]:
pp2.fit(X_train,y_train)

Pipeline(steps=[('bow', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('classifier', MultinomialNB())])

In [13]:
predictions2=pp2.predict(X_test)

In [14]:
print(confusion_matrix(y_test, predictions2))
print('\n')
print(classification_report(y_test, predictions2))

[[ 837  855]
 [ 292 1481]]


              precision    recall  f1-score   support

        fake       0.74      0.49      0.59      1692
        real       0.63      0.84      0.72      1773

    accuracy                           0.67      3465
   macro avg       0.69      0.66      0.66      3465
weighted avg       0.69      0.67      0.66      3465



In [15]:
#passive-aggresive classifier
from sklearn.linear_model import PassiveAggressiveClassifier
pp3=Pipeline([
    ('bow',CountVectorizer()),
    ('tfidf',TfidfTransformer()),
    ('classifier',PassiveAggressiveClassifier())
    ])

In [16]:
pp3.fit(X_train,y_train)

Pipeline(steps=[('bow', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('classifier', PassiveAggressiveClassifier())])

In [17]:
predictions3=pp3.predict(X_test)

In [18]:
print(confusion_matrix(y_test, predictions3))
print('\n')
print(classification_report(y_test, predictions3))

[[1118  574]
 [ 557 1216]]


              precision    recall  f1-score   support

        fake       0.67      0.66      0.66      1692
        real       0.68      0.69      0.68      1773

    accuracy                           0.67      3465
   macro avg       0.67      0.67      0.67      3465
weighted avg       0.67      0.67      0.67      3465



In [19]:
#logistic regression
pp4=Pipeline([
    ('bow',CountVectorizer()),
    ('tfidf',TfidfTransformer()),
    ('classifier',LogisticRegression())
    ])

In [20]:
pp4.fit(X_train,y_train)

Pipeline(steps=[('bow', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('classifier', LogisticRegression())])

In [21]:
predictions4=pp4.predict(X_test)

In [22]:
print(confusion_matrix(y_test, predictions4))
print('\n')
print(classification_report(y_test, predictions4))

[[1086  606]
 [ 406 1367]]


              precision    recall  f1-score   support

        fake       0.73      0.64      0.68      1692
        real       0.69      0.77      0.73      1773

    accuracy                           0.71      3465
   macro avg       0.71      0.71      0.71      3465
weighted avg       0.71      0.71      0.71      3465



In [58]:
#SVM was taking a lot of time with not significant value

# BASIC MODELLING - Statement

In [23]:
X=df['statement']
y=df['label_fnn']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [24]:
#Random Forest Classifier
pp=Pipeline([
    ('bow',CountVectorizer()),
    ('tfidf',TfidfTransformer()),
    ('classifier',RandomForestClassifier())
    ])

In [25]:
pp.fit(X_train,y_train)

Pipeline(steps=[('bow', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('classifier', RandomForestClassifier())])

In [26]:
predictions=pp.predict(X_test)

In [27]:
print(confusion_matrix(y_test, predictions))
print('\n')
print(classification_report(y_test, predictions))

[[1041  651]
 [ 592 1181]]


              precision    recall  f1-score   support

        fake       0.64      0.62      0.63      1692
        real       0.64      0.67      0.66      1773

    accuracy                           0.64      3465
   macro avg       0.64      0.64      0.64      3465
weighted avg       0.64      0.64      0.64      3465



In [28]:
#Multinomial Naive Bayes
pp2=Pipeline([
    ('bow',CountVectorizer()),
    ('tfidf',TfidfTransformer()),
    ('classifier',MultinomialNB())
    ])

In [29]:
pp2.fit(X_train,y_train)

Pipeline(steps=[('bow', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('classifier', MultinomialNB())])

In [30]:
predictions2=pp2.predict(X_test)

In [31]:
print(confusion_matrix(y_test, predictions2))
print('\n')
print(classification_report(y_test, predictions2))

[[ 887  805]
 [ 420 1353]]


              precision    recall  f1-score   support

        fake       0.68      0.52      0.59      1692
        real       0.63      0.76      0.69      1773

    accuracy                           0.65      3465
   macro avg       0.65      0.64      0.64      3465
weighted avg       0.65      0.65      0.64      3465



In [32]:
#passive-aggresive classifier
from sklearn.linear_model import PassiveAggressiveClassifier
pp3=Pipeline([
    ('bow',CountVectorizer()),
    ('tfidf',TfidfTransformer()),
    ('classifier',PassiveAggressiveClassifier())
    ])

In [33]:
pp3.fit(X_train,y_train)

Pipeline(steps=[('bow', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('classifier', PassiveAggressiveClassifier())])

In [34]:
predictions3=pp3.predict(X_test)

In [35]:
print(confusion_matrix(y_test, predictions3))
print('\n')
print(classification_report(y_test, predictions3))

[[ 955  737]
 [ 697 1076]]


              precision    recall  f1-score   support

        fake       0.58      0.56      0.57      1692
        real       0.59      0.61      0.60      1773

    accuracy                           0.59      3465
   macro avg       0.59      0.59      0.59      3465
weighted avg       0.59      0.59      0.59      3465



In [36]:
#logistic regression
pp4=Pipeline([
    ('bow',CountVectorizer()),
    ('tfidf',TfidfTransformer()),
    ('classifier',LogisticRegression())
    ])

In [37]:
pp4.fit(X_train,y_train)

Pipeline(steps=[('bow', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('classifier', LogisticRegression())])

In [38]:
predictions4=pp4.predict(X_test)

In [39]:
print(confusion_matrix(y_test, predictions4))
print('\n')
print(classification_report(y_test, predictions4))

[[1058  634]
 [ 599 1174]]


              precision    recall  f1-score   support

        fake       0.64      0.63      0.63      1692
        real       0.65      0.66      0.66      1773

    accuracy                           0.64      3465
   macro avg       0.64      0.64      0.64      3465
weighted avg       0.64      0.64      0.64      3465



# BASIC MODELLING - Statement+full text

In [40]:
df['all'] = df['fullText_based_content'] + " " + df['statement']


In [41]:
X=df['all']
y=df['label_fnn']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [42]:
#Random Forest Classifier
pp=Pipeline([
    ('bow',CountVectorizer()),
    ('tfidf',TfidfTransformer()),
    ('classifier',RandomForestClassifier())
    ])

In [43]:
pp.fit(X_train,y_train)

Pipeline(steps=[('bow', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('classifier', RandomForestClassifier())])

In [44]:
predictions=pp.predict(X_test)

In [45]:
print(confusion_matrix(y_test, predictions))
print('\n')
print(classification_report(y_test, predictions))

[[1022  670]
 [ 414 1359]]


              precision    recall  f1-score   support

        fake       0.71      0.60      0.65      1692
        real       0.67      0.77      0.71      1773

    accuracy                           0.69      3465
   macro avg       0.69      0.69      0.68      3465
weighted avg       0.69      0.69      0.68      3465



In [46]:
#Multinomial Naive Bayes
pp2=Pipeline([
    ('bow',CountVectorizer()),
    ('tfidf',TfidfTransformer()),
    ('classifier',MultinomialNB())
    ])

In [47]:
pp2.fit(X_train,y_train)

Pipeline(steps=[('bow', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('classifier', MultinomialNB())])

In [48]:
predictions2=pp2.predict(X_test)

In [49]:
print(confusion_matrix(y_test, predictions2))
print('\n')
print(classification_report(y_test, predictions2))

[[ 835  857]
 [ 293 1480]]


              precision    recall  f1-score   support

        fake       0.74      0.49      0.59      1692
        real       0.63      0.83      0.72      1773

    accuracy                           0.67      3465
   macro avg       0.69      0.66      0.66      3465
weighted avg       0.69      0.67      0.66      3465



In [50]:
#passive-aggresive classifier
from sklearn.linear_model import PassiveAggressiveClassifier
pp3=Pipeline([
    ('bow',CountVectorizer()),
    ('tfidf',TfidfTransformer()),
    ('classifier',PassiveAggressiveClassifier())
    ])

In [51]:
pp3.fit(X_train,y_train)

Pipeline(steps=[('bow', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('classifier', PassiveAggressiveClassifier())])

In [52]:
predictions3=pp3.predict(X_test)

In [53]:
print(confusion_matrix(y_test, predictions3))
print('\n')
print(classification_report(y_test, predictions3))

[[1137  555]
 [ 562 1211]]


              precision    recall  f1-score   support

        fake       0.67      0.67      0.67      1692
        real       0.69      0.68      0.68      1773

    accuracy                           0.68      3465
   macro avg       0.68      0.68      0.68      3465
weighted avg       0.68      0.68      0.68      3465



In [54]:
#logistic regression
pp4=Pipeline([
    ('bow',CountVectorizer()),
    ('tfidf',TfidfTransformer()),
    ('classifier',LogisticRegression())
    ])

In [55]:
pp4.fit(X_train,y_train)

Pipeline(steps=[('bow', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('classifier', LogisticRegression())])

In [56]:
predictions4=pp4.predict(X_test)

In [57]:
print(confusion_matrix(y_test, predictions4))
print('\n')
print(classification_report(y_test, predictions4))

[[1089  603]
 [ 408 1365]]


              precision    recall  f1-score   support

        fake       0.73      0.64      0.68      1692
        real       0.69      0.77      0.73      1773

    accuracy                           0.71      3465
   macro avg       0.71      0.71      0.71      3465
weighted avg       0.71      0.71      0.71      3465



### BERT MODEL - Full Text based Content - implemented in Kaggle (Needed TPU)